In [ ]:
import torch
import torch.nn as nn
import random

# ===== 1. DỮ LIỆU =====
chars = list("abcdefghijklmnopqrstuvwxyz")
vocab = ['<pad>', '<sos>', '<eos>'] + chars
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for ch, i in stoi.items()}

def random_string(min_len=3, max_len=6):
    L = random.randint(min_len, max_len)
    return ''.join(random.choices(chars, k=L))

def encode(seq):
    return [stoi['<sos>']] + [stoi[c] for c in seq] + [stoi['<eos>']]

def pad_batch(seqs):
    max_len = max(len(s) for s in seqs)
    pad_idx = stoi['<pad>']
    padded = [s + [pad_idx]*(max_len-len(s)) for s in seqs]
    mask = [[1 if t != pad_idx else 0 for t in s] for s in padded]
    return torch.tensor(padded), torch.tensor(mask)

# Tạo dữ liệu train
data = [random_string() for _ in range(3000)]
inputs = [encode(s) for s in data]
targets = [encode(s[::-1]) for s in data]


# ===== 2. MÔ HÌNH =====
class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)

    def forward(self, x, mask):
        emb = self.embed(x)
        lengths = mask.sum(1).cpu()
        packed = nn.utils.rnn.pack_padded_sequence(emb, lengths, batch_first=True, enforce_sorted=False)
        _, hidden = self.rnn(packed)
        return hidden  # (1, batch, hidden)

class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, tgt, hidden):
        emb = self.embed(tgt)
        out, _ = self.rnn(emb, hidden)
        logits = self.fc(out)
        return logits

class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.encoder = Encoder(vocab_size, hidden_dim)
        self.decoder = Decoder(vocab_size, hidden_dim)

    def forward(self, src, src_mask, tgt):
        hidden = self.encoder(src, src_mask)
        logits = self.decoder(tgt[:, :-1], hidden)  # bỏ token <eos>
        return logits


# ===== 3. HUẤN LUYỆN =====
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Seq2Seq(len(vocab), 64).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=stoi['<pad>'])

batch_size = 64

for epoch in range(200):
    model.train()
    total_loss = 0
    # shuffle dữ liệu
    order = torch.randperm(len(inputs))
    for i in range(0, len(inputs), batch_size):
        idx = order[i:i+batch_size]
        src_batch = [inputs[j] for j in idx]
        tgt_batch = [targets[j] for j in idx]
        src, src_mask = pad_batch(src_batch)
        tgt, _ = pad_batch(tgt_batch)
        src, src_mask, tgt = src.to(device), src_mask.to(device), tgt.to(device)

        logits = model(src, src_mask, tgt)
        loss = criterion(logits.reshape(-1, len(vocab)), tgt[:, 1:].reshape(-1))

        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1:03d}, loss={total_loss/len(inputs):.4f}")

# ===== 4. SUY LUẬN (INFERENCE) =====
def predict(seq):
    model.eval()
    src = torch.tensor([encode(seq)], device=device)
    src_mask = torch.ones_like(src)
    hidden = model.encoder(src, src_mask)

    dec_input = torch.tensor([[stoi['<sos>']]], device=device)
    result = ''
    for _ in range(10):
        logits = model.decoder(dec_input, hidden)
        next_token = logits[:, -1].argmax(1).item()
        ch = itos[next_token]
        if ch == '<eos>': break
        result += ch
        dec_input = torch.cat([dec_input, torch.tensor([[next_token]], device=device)], dim=1)
    return result

print("\n=== TEST ===")
for s in ["abc", "hello", "xyz", "chat"]:
    print(f"{s} -> {predict(s)}")

# ===== 5. LƯU MÔ HÌNH =====
torch.save(model.state_dict(), "seq2seq_reverse.pt")
print("Đã lưu mô hình vào file seq2seq_reverse.pt")


Epoch 001, loss=0.0486
Epoch 002, loss=0.0409
Epoch 003, loss=0.0340
Epoch 004, loss=0.0286
Epoch 005, loss=0.0242
Epoch 006, loss=0.0204
Epoch 007, loss=0.0171
Epoch 008, loss=0.0144
Epoch 009, loss=0.0122
Epoch 010, loss=0.0104
Epoch 011, loss=0.0089
Epoch 012, loss=0.0077
Epoch 013, loss=0.0067
Epoch 014, loss=0.0059
Epoch 015, loss=0.0051
Epoch 016, loss=0.0046
Epoch 017, loss=0.0041
Epoch 018, loss=0.0037
Epoch 019, loss=0.0032
Epoch 020, loss=0.0030
Epoch 021, loss=0.0026
Epoch 022, loss=0.0024
Epoch 023, loss=0.0023
Epoch 024, loss=0.0021
Epoch 025, loss=0.0020
Epoch 026, loss=0.0018
Epoch 027, loss=0.0017
Epoch 028, loss=0.0015
Epoch 029, loss=0.0013
Epoch 030, loss=0.0013
Epoch 031, loss=0.0012
Epoch 032, loss=0.0011
Epoch 033, loss=0.0011
Epoch 034, loss=0.0010
Epoch 035, loss=0.0010
Epoch 036, loss=0.0011
Epoch 037, loss=0.0011
Epoch 038, loss=0.0010
Epoch 039, loss=0.0009
Epoch 040, loss=0.0008
Epoch 041, loss=0.0007
Epoch 042, loss=0.0006
Epoch 043, loss=0.0008
Epoch 044, 

In [ ]:
# ===== 6. TẢI LẠI MÔ HÌNH (BỎ QUA TRAIN) =====
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Seq2Seq(len(vocab), 64).to(device)
model.load_state_dict(torch.load("seq2seq_reverse.pt", map_location=device))
model.eval()
print("Đã tải mô hình huấn luyện sẵn")

Đã tải mô hình huấn luyện sẵn


In [ ]:
predict('pyth')

'htyp'